In [ ]:
from collections import defaultdict, Counter
from string import ascii_uppercase
from os import path
import os
import shutil
import json

from metadata import load_csv, zfilter

import pandas as pd
from numpy import random
import numpy as np

random.seed(123456)

%matplotlib inline

In [ ]:
supergroups = {
    'indica':               'Indica',
    'japonica':             'Japonica',
    'temperate japonica':   'Japonica',
    'tropical japonica':    'Japonica',
}

In [ ]:
allruns = load_csv("all_25228runs_raw.csv")

In [ ]:
allpd = pd.DataFrame(allruns)
allpd['group'] = allpd.group.str.lower()
allpd['supergroup'] = [supergroups.get(g, 'other') for g in allpd.group]

In [ ]:
indjap = allpd.query('supergroup in ["Indica", "Japonica"]')

print("Groups:")
for g, c in Counter(list(indjap.group)).most_common():
    print("  -", g, c)
print("Supergroups:")
for g, c in Counter(list(indjap.supergroup)).most_common():
    print("  -", g, c)
   

In [ ]:
# Remove all samples that weren't sequenced as 6 runs
indjap = indjap.groupby('sra_sample').filter(lambda x: len(x) == 6)
print("Select 6s:", len(indjap))

# Remove all runs outside 1 sd from mean number of reads
indjap = zfilter(indjap, 'num_reads', 1)
print("Z-score filtering:", len(indjap))
# Remove all samps with fewer than 6 remaining runs
indjap = indjap.groupby('sra_sample').filter(lambda x: len(x) == 6)
print("6s again:", len(indjap))

In [ ]:
indjap.groupby('supergroup').supergroup.count()

In [ ]:
groups = defaultdict(list)
sampgrp = {}
for group, runs in indjap.groupby('supergroup'):
    for samp in runs.sra_sample.unique():
        groups[group].append(samp)
        sampgrp[samp] = group

In [ ]:
def genset(groups,  each=8):
    sample = set()
    for group, items in groups.items():
        while len(sample) < each:
            sample.add(random.choice(items))
    return tuple(sample)

In [ ]:
outdir = "sets/96s"
try:
    shutil.rmtree(outdir)
except Exception as e:
    print(str(e))
os.mkdir(outdir)

In [ ]:
ngroup = 100
allruns = set()
allsets = set()

i = 0
while i < 100:
    samples = genset(groups, each=8)
    if samples in allsets:
        continue
    i += 1
    allsets.add(samples)
    label = "{0:0{width}d}".format(i + 1, width=int(np.ceil(np.log10(ngroup + 1))))
    outfile = path.join(outdir, "3krice_set_{}.txt".format(label))
    jsonf = outfile + '.json'
    setjson = defaultdict(dict)
    setruns = []
    for sample in samples:
        sampruns = []
        for run in indjap.query('sra_sample == "{}"'.format(sample)).sra_id:
            sampruns.append(run)
            setruns.append(run)
            allruns.add(run)
        setjson[sampgrp[samp]][sample] = sampruns
    with open(outfile, 'w') as fh:
        for run in setruns:
            print(run, file=fh)

with open(path.join(outdir, "3krice_set_ALL.txt"), 'w') as fh:
    for run in sorted(allruns):
        print(run, file=fh)

In [ ]:
len(allruns)